# 4. Getting financial data (API)

Daniel Ruiz, MSc in Data Science and Business Analytics (DSBA), Bocconi University

Reference codes (alphabetically):
    
    - https://github.com/addisonlynch/iexfinance/blob/master/README.rst
    
# USA = IEX
## 4.1. Examples

from iexfinance.stocks import Stock

mystock = Stock("TSLA")

mystock.get_quote()

## 4.2. Getting the data

In [2]:
# import packages
import csv
from datetime import datetime
from iexfinance.stocks import get_historical_intraday, get_historical_data

# set companies, dates
my_companies = [['us_abercrombie','ANF'],
                ['us_boeing','BA'],
                ['us_beyondmeat','BYND'],
                ['us_morganstanley','MS'],
                ['us_jpmorgan', 'JPM'],
                ['us_exxonmobil','XOM'],
                ['us_americanair','AAL'],
                ['us_cocacola','KO'],
                ['us_tesla','TSLA'],
                ['us_wsj','NWSA']]
start_date = datetime(2020, 1, 1)
end_date = datetime.now().date()

### 4.2.1 Intraday

for company in my_companies:
    
    # Open/Create a file to append data
    
    minutes = get_historical_intraday(company[1], date)

    with open("Dataset_IEX_01\\"+company[0]+".csv", 'a', encoding="utf-8") as csvFile:
    
        # Use csv writer
        csvWriter = csv.writer(csvFile)
        
        for minute in minutes:
            csvWriter.writerow(list(minute.values()))

### 4.2.1 Daily

In [3]:
for company in my_companies:
    
    days = get_historical_data(company[1],start_date,end_date)
    print(company[0])

    with open("Dataset_API_Daily\\"+company[0]+".csv", 'w', encoding="utf-8") as csvFile:
    
        # Use csv writer
        csvWriter = csv.writer(csvFile)
        
        for day in days:
            # ['date', 'open', 'high', 'low', 'close', 'volume']
            csvWriter.writerow([day]+list(days[day].values()))

us_abercrombie
us_boeing
us_beyondmeat
us_morganstanley
us_jpmorgan
us_exxonmobil
us_americanair
us_cocacola
us_tesla
us_wsj


# BRA = PYBOVESPA
## 4.3. Examples
http://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/historico/mercado-a-vista/series-historicas/

In [5]:
# import packages
import csv
import pandas as pd

# load saved dataset
all_data = pd.read_csv('COTAHIST_A2020.txt',names=['raw'])
all_data = all_data['raw'].tolist()

# code positions
pos0 = [1,3,11,13,25,28,40,50,53,57,70,83,96,109,122,135,148,153,171,189,202,203,211,218,231,243]
pos0 = [pos-1 for pos in pos0]

days_2020=[]

# extract desired variables
for data in all_data:
    
    sup=[]
    for i in range(1,len(pos0)):
        sup.append(data[pos0[i-1]:pos0[i]])
    
    if sup[1]=='COTAHIST':
        continue
    else:
        # date, ticker, open, max, min, close,volume
        sup = [sup[1],
               sup[3].replace(" ",""),
               int(sup[9]),
               int(sup[10]),
               int(sup[11]),
               int(sup[13]),
               int(sup[18])]
        
        sup[0]=sup[0][:4]+'-'+sup[0][4:6]+'-'+sup[0][6:8]
                   
        days_2020.append(sup)
        
# filter desired companies
my_companies = [['br_embraer','EMBR3'],
                ['br_americanas','LAME4'],
                ['br_pontofrio','VVAR3'],
                ['br_petrobras','PETR4'],
                ['br_bradesco','BBDC4'],
                ['br_itau','ITUB4'],
                ['br_renner','LREN3'],
                ['br_gol','GOLL4'],
                ['br_magazineluiza','MGLU3']]

keeps=[]
dict_comp=dict(my_companies)
dict_comp = {v: k for k, v in dict_comp.items()}

for day in days_2020:
    if day[1] in dict_comp:
        keeps.append(day)

for keep in keeps:
    keep[1]=dict_comp[keep[1]]
    
    
stocks = pd.DataFrame(keeps)
stocks.columns = ['date','company','open','high','low','close','volume']
stocks = stocks[['company','date', 'open', 'high', 'low', 'close', 'volume']]
stocks = stocks.sort_values(['company','date'])

for company in my_companies:

    sup = stocks[stocks.company==company[0]]

    with open("Dataset_API_Daily\\"+company[0]+".csv", 'w', encoding="utf-8") as csvFile:
    
        # Use csv writer
        csvWriter = csv.writer(csvFile)
        days = sup.values.tolist()
        
        for day in days:
            csvWriter.writerow(day[1:])